In [ ]:
import os
import random
import shutil

def split_dataset(dataset_path, part1_path, part2_path):
    # Create directories if they don't exist
    os.makedirs(part1_path, exist_ok=True)
    os.makedirs(part2_path, exist_ok=True)

    # List all images in the dataset
    images = [f for f in os.listdir(dataset_path) if os.path.isfile(os.path.join(dataset_path, f))]
    random.shuffle(images)

    # Split the dataset into two equal parts
    split_index = len(images) // 2
    part1_images = images[:split_index]
    part2_images = images[split_index:]

    # Move images to their respective directories
    for img in part1_images:
        shutil.move(os.path.join(dataset_path, img), os.path.join(part1_path, img))
    for img in part2_images:
        shutil.move(os.path.join(dataset_path, img), os.path.join(part2_path, img))

    print(f"Dataset split into {part1_path} and {part2_path}")

# Example usage
split_dataset('Datasets\places2', 'Datasets/split1', 'Datasets/split2')

Dataset split into Github/inpainting_and_forgery_detection/split1 and Github/inpainting_and_forgery_detection/split2


In [1]:
pip install matplotlib pandas numpy 


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install opencv-python


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!python -m ipykernel install --user --name=your_env --display-name "Python (venv)"


Installed kernelspec your_env in C:\Users\revna\AppData\Roaming\jupyter\kernels\your_env


In [4]:
import PIL
import numpy
import matplotlib.pyplot
print("All imports work!")

All imports work!


In [5]:
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm import tqdm  # For progress tracking

def create_mask(image_shape, min_mask_percentage=0.10, max_mask_percentage=0.30):
    """
    Create a more dynamic mask with controlled total masked area between min and max percentages
    
    Args:
    - image_shape: Shape of the image (height, width)
    - min_mask_percentage: Minimum percentage of image to be masked (default 10%)
    - max_mask_percentage: Maximum percentage of image to be masked (default 30%)
    
    Returns:
    - mask: Numpy array with mask (255 for masked regions)
    """
    height, width = image_shape[:2]
    total_pixels = height * width
    
    # Ensure min is not greater than max
    min_mask_percentage = min(min_mask_percentage, max_mask_percentage)
    
    # Calculate target masked pixels within the range
    min_masked_pixels = int(total_pixels * min_mask_percentage)
    max_masked_pixels = int(total_pixels * max_mask_percentage)
    target_masked_pixels = np.random.randint(min_masked_pixels, max_masked_pixels + 1)
    
    # Initialize mask
    mask = np.zeros((height, width), dtype=np.uint8)
    
    # Randomize mask generation approach
    mask_type = np.random.choice(['rectangle', 'multiple', 'irregular'])
    
    if mask_type == 'rectangle':
        # Single rectangular region
        max_side = int(np.sqrt(target_masked_pixels))
        rect_width = np.random.randint(max_side // 2, max_side)
        rect_height = target_masked_pixels // rect_width
        
        x = np.random.randint(0, width - rect_width)
        y = np.random.randint(0, height - rect_height)
        
        mask[y:y+rect_height, x:x+rect_width] = 255
    
    elif mask_type == 'multiple':
        # Multiple smaller masked regions
        num_regions = np.random.randint(2, 6)
        pixels_per_region = target_masked_pixels // num_regions
        
        for _ in range(num_regions):
            region_size = int(np.sqrt(pixels_per_region))
            x = np.random.randint(0, width - region_size)
            y = np.random.randint(0, height - region_size)
            
            mask[y:y+region_size, x:x+region_size] = 255
    
    else:  # irregular mask
        # Create an irregular mask using random walk
        current_masked_pixels = 0
        max_attempts = 1000
        attempts = 0
        
        while current_masked_pixels < target_masked_pixels and attempts < max_attempts:
            # Random walk
            x, y = np.random.randint(0, width), np.random.randint(0, height)
            step_size = np.random.randint(1, 10)
            
            # Ensure we don't go out of bounds
            x = max(0, min(x, width-1))
            y = max(0, min(y, height-1))
            
            # Create a small irregular region
            region = mask[max(0, y-step_size):min(height, y+step_size),
                          max(0, x-step_size):min(width, x+step_size)]
            
            # Add region to mask if not already masked
            new_mask_pixels = np.sum(region == 0)
            if new_mask_pixels > 0:
                region[region == 0] = 255
                current_masked_pixels += new_mask_pixels
            
            attempts += 1
    
    # Verify mask percentage (optional - can be commented out for bulk processing)
    # masked_percentage = np.sum(mask == 255) / total_pixels
    # print(f"Mask Type: {mask_type}, Masked Percentage: {masked_percentage:.2%}, " 
    #       f"Range: {min_mask_percentage:.2%}-{max_mask_percentage:.2%}")
    
    return mask


def inpaint_image(image_path, output_dir, min_mask_percentage=0.05, max_mask_percentage=0.15, save_visualization=False):
    """
    Inpaint an image using OpenCV's Telea inpainting method
    
    Args:
    - image_path: Path to input image
    - output_dir: Directory to save output images
    - min_mask_percentage: Minimum percentage of image to be masked
    - max_mask_percentage: Maximum percentage of image to be masked
    - save_visualization: Whether to save the visualization (False for bulk processing)
    """
    # Create output directories
    os.makedirs(output_dir, exist_ok=True)
    if save_visualization:
        os.makedirs(os.path.join(output_dir, 'visualizations'), exist_ok=True)
    
    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Warning: Couldn't read image {image_path}")
        return
    
    # Create a mask with specified min/max percentage
    mask = create_mask(image.shape, min_mask_percentage, max_mask_percentage)
    
    # Telea Method
    inpainted_telea = cv2.inpaint(image, mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)
    
    # Save inpainted image
    filename = os.path.basename(image_path)
    cv2.imwrite(os.path.join(output_dir, f'telea_{filename}'), inpainted_telea)
    
    # Save visualization only if requested (disabled for bulk processing)
    if save_visualization:
        # Create visualization figure
        plt.figure(figsize=(15, 5))
        
        # Original Image
        plt.subplot(1, 3, 1)
        plt.title('Original Image')
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        
        # Mask Visualization
        plt.subplot(1, 3, 2)
        plt.title('Mask')
        plt.imshow(mask, cmap='gray')
        plt.axis('off')
        
        # Telea Inpainting
        plt.subplot(1, 3, 3)
        plt.title('Telea Inpainting')
        plt.imshow(cv2.cvtColor(inpainted_telea, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        
        # Save visualization
        vis_path = os.path.join(output_dir, 'visualizations', f'inpainting_{filename}')
        plt.tight_layout()
        plt.savefig(vis_path)
        plt.close()


def process_directory(input_dir, output_dir, max_images=10000, min_mask_percentage=0.05, max_mask_percentage=0.15):
    """
    Process up to max_images images in a directory
    
    Args:
    - input_dir: Directory with input images
    - output_dir: Directory to save output images
    - max_images: Maximum number of images to process
    - min_mask_percentage: Minimum percentage of image to be masked
    - max_mask_percentage: Maximum percentage of image to be masked
    """
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Get list of all image files
    image_files = []
    for root, _, files in os.walk(input_dir):
        for filename in files:
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                image_files.append(os.path.join(root, filename))
    
    # Limit to max_images
    total_images = min(len(image_files), max_images)
    if len(image_files) < max_images:
        print(f"Warning: Only found {len(image_files)} images, fewer than the requested {max_images}")
    else:
        print(f"Processing {total_images} images...")
    
    # Process images with progress bar
    for image_path in tqdm(image_files[:total_images], desc="Inpainting Images"):
        try:
            # Disable visualization for bulk processing to speed up
            inpaint_image(
                image_path=image_path,
                output_dir=output_dir,
                min_mask_percentage=min_mask_percentage,
                max_mask_percentage=max_mask_percentage,
                save_visualization=True  
            )
        except Exception as e:
            print(f"Error processing {image_path}: {str(e)}")


def main():
    # Process images
    process_directory(
        input_dir='Datasets/split2',  # Input image directory
        output_dir='Datasets/in_split2',  # Output directory
        max_images=5000,  # Process up to 5,000 images
        min_mask_percentage=0.05,  # Minimum 5% of image will be masked
        max_mask_percentage=0.15   # Maximum 15% of image will be masked
    )

if __name__ == '__main__':
    main()

Processing 5000 images...


Inpainting Images: 100%|██████████| 5000/5000 [20:46<00:00,  4.01it/s]


In [10]:
import os
import random
import shutil

def split_train_test(shuffled_dataset_dir, train_dir, test_dir):
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)
    
    images = [f for f in os.listdir(shuffled_dataset_dir) if os.path.isfile(os.path.join(shuffled_dataset_dir, f))]
    random.shuffle(images)
    
    split_index = len(images) // 2
    train_images = images[:split_index]
    test_images = images[split_index:]
    
    for img in train_images:
        shutil.move(os.path.join(shuffled_dataset_dir, img), os.path.join(train_dir, img))
    for img in test_images:
        shutil.move(os.path.join(shuffled_dataset_dir, img), os.path.join(test_dir, img))
    
    print(f"Dataset split into training and testing sets")

# Example usage
split_train_test("C:/Users/revna/Downloads/datasets/datasets/in_split2", 'Datasets/6000/Train/1', 'Datasets/6000/Test/1')

Dataset split into training and testing sets


In [11]:
%pip install keras

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
%pip install h5py

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
%pip install keras

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
%pip install seaborn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, LeakyReLU, BatchNormalization, Reshape, Conv2DTranspose, Input, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

def load_and_preprocess_data(train_dir, test_dir, img_height=128, img_width=128, batch_size=32):
    """
    Load and preprocess training and testing data
    """
    # Data augmentation for training
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    
    # Only rescaling for testing
    test_datagen = ImageDataGenerator(rescale=1./255)
    
    # Load data from directory
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary',
        color_mode='rgb'
    )
    
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary',
        color_mode='rgb',
        shuffle=False
    )
    
    return train_generator, test_generator

def build_discriminator(input_shape=(128, 128, 3)):
    """
    Build the discriminator model that will classify real vs fake images
    """
    discriminator = Sequential([
        Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=input_shape),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        
        Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        
        Conv2D(256, (5, 5), strides=(2, 2), padding='same'),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        
        Conv2D(512, (5, 5), strides=(2, 2), padding='same'),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    
    discriminator.compile(loss='binary_crossentropy', optimizer=Adam(1e-4), metrics=['accuracy'])
    return discriminator

def build_generator(latent_dim=100, output_shape=(128, 128, 3)):
    """
    Build the generator model that will create fake images
    """
    generator = Sequential([
        Dense(8*8*512, input_shape=(latent_dim,)),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Reshape((8, 8, 512)),
        
        Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        
        Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        
        Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        
        Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', activation='tanh')
    ])
    
    return generator

def build_gan(generator, discriminator):
    """
    Connect the generator and discriminator to form the GAN
    """
    discriminator.trainable = False
    gan_input = Input(shape=(100,))
    fake_image = generator(gan_input)
    gan_output = discriminator(fake_image)
    gan = Model(gan_input, gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=Adam(1e-4))
    return gan

def build_classifier():
    """
    Build a classifier for forgery detection
    """
    classifier = Sequential([
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(128, 128, 3)),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    
    classifier.compile(loss='binary_crossentropy', optimizer=Adam(1e-4), metrics=['accuracy'])
    return classifier

def train_gan_and_classifier(generator, discriminator, gan, classifier, train_generator, test_generator, 
                         latent_dim=100, epochs=50, batch_size=32, save_interval=10):
    """
    Train both the GAN and the classifier
    """
    # Create directories for saving generated images
    if not os.path.exists("C:/Users/revna/Desktop/Github/Inpainting_and_Forgery_Detection/gan_generated_images/35_2500"):
        os.makedirs("C:/Users/revna/Desktop/Github/Inpainting_and_Forgery_Detection/gan_generated_images/35_2500")
    
    # Get number of batches per epoch
    batches_per_epoch = len(train_generator)
    
    # Training history
    d_losses, g_losses, c_losses = [], [], []
    
    # Train the models
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        epoch_d_losses, epoch_g_losses, epoch_c_losses = [], [], []
        
        for batch in range(batches_per_epoch):
            # Get a batch of real images
            real_images, real_labels = next(train_generator)
            batch_size = real_images.shape[0]
            
            # Create noise for generator input
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            
            # Generate a batch of fake images
            generated_images = generator.predict(noise)
            
            # Train the discriminator on real and fake images
            d_loss_real = discriminator.train_on_batch(real_images, np.ones((batch_size, 1)))
            d_loss_fake = discriminator.train_on_batch(generated_images, np.zeros((batch_size, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            
            # Train the generator
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))
            
            # Train the classifier on the real images with their labels
            c_loss = classifier.train_on_batch(real_images, real_labels)
            
            # Print progress
            if batch % 20 == 0:
                print(f"Batch {batch}/{batches_per_epoch}, D Loss: {d_loss[0]}, G Loss: {g_loss}, C Loss: {c_loss[0]}")
            
            epoch_d_losses.append(d_loss[0])
            epoch_g_losses.append(g_loss)
            epoch_c_losses.append(c_loss[0])
        
        # Compute the average losses for this epoch
        d_losses.append(np.mean(epoch_d_losses))
        g_losses.append(np.mean(epoch_g_losses))
        c_losses.append(np.mean(epoch_c_losses))
        
        # Generate and save sample images
        if epoch % save_interval == 0:
            sample_noise = np.random.normal(0, 1, (16, latent_dim))
            generated_images = generator.predict(sample_noise)
            
            # Rescale from [-1, 1] to [0, 1]
            generated_images = 0.5 * generated_images + 0.5
            
            fig, axs = plt.subplots(4, 4, figsize=(10, 10))
            count = 0
            for i in range(4):
                for j in range(4):
                    axs[i, j].imshow(generated_images[count])
                    axs[i, j].axis('off')
                    count += 1
            fig.savefig(f"gan_generated_images/35_2500/epoch_{epoch}.png")
            plt.close()
    
    # Plot the losses
    plt.figure(figsize=(10, 6))
    plt.plot(d_losses, label='Discriminator')
    plt.plot(g_losses, label='Generator')
    plt.plot(c_losses, label='Classifier')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('loss_history.png')
    plt.close()
    
    return d_losses, g_losses, c_losses

def evaluate_classifier(classifier, test_generator):
    """
    Evaluate the classifier on the test set
    """
    # Get test data
    test_generator.reset()
    y_true = test_generator.classes
    
    # Predict on test data
    steps = len(test_generator)
    predictions = classifier.predict(test_generator, steps=steps)
    y_pred = (predictions > 0.5).astype(int).flatten()
    
    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    # Create confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Forgery (0)', 'Authentic (1)'],
                yticklabels=['Forgery (0)', 'Authentic (1)'])
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.savefig('confusion_matrix.png')
    plt.close()
    
    # Print metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    
    return accuracy, precision, recall, f1, cm

def main():
    # Configuration
    img_height, img_width = 128, 128
    batch_size = 16
    latent_dim = 100
    epochs = 35
    
    # Define directories for your data
    # Replace these with your actual data paths
    train_dir = "C:/Users/revna/Desktop/Github/Inpainting_and_Forgery_Detn/Datasets/2500/train"  # Should have subdirectories for each class (0 and 1)
    test_dir = "C:/Users/revna/Desktop/Github/Inpainting_and_Forgery_Detn/Datasets/2500/test"    # Should have subdirectories for each class (0 and 1)
    
    # Load and preprocess data
    train_generator, test_generator = load_and_preprocess_data(
        train_dir, test_dir, img_height, img_width, batch_size
    )
    
    # Build models
    discriminator = build_discriminator((img_height, img_width, 3))
    generator = build_generator(latent_dim, (img_height, img_width, 3))
    gan = build_gan(generator, discriminator)
    classifier = build_classifier()
    
    # Train models
    train_gan_and_classifier(
        generator, discriminator, gan, classifier,
        train_generator, test_generator,
        latent_dim, epochs, batch_size
    )
    
    # Evaluate classifier
    evaluate_classifier(classifier, test_generator)
    
    # Save models
    generator.save('Models/35/2500/generator_model.h5')
    discriminator.save('Models/35/2500/discriminator_model.h5')
    classifier.save('Models/35/2500/classifier_model.h5')

if __name__ == "__main__":
    main()

Found 1369 images belonging to 2 classes.
Found 1319 images belonging to 2 classes.


c:\Users\revna\Desktop\Github\Inpainting_and_Forgery_Detection\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\revna\Desktop\Github\Inpainting_and_Forgery_Detection\venv\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
c:\Users\revna\Desktop\Github\Inpainting_and_Forgery_Detection\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/35
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 661ms/step


c:\Users\revna\Desktop\Github\Inpainting_and_Forgery_Detection\venv\Lib\site-packages\keras\src\backend\tensorflow\trainer.py:82: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Batch 0/86, D Loss: 0.6885969638824463, G Loss: 0.6880049109458923, C Loss: 0.6973273158073425
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
Batch 20/86, D Loss: 0.6909373998641968, G Loss: 0.5668201446533203, C Loss: 0.7085356712341309
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 1

c:\Users\revna\Desktop\Github\Inpainting_and_Forgery_Detection\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


83/83 ━━━━━━━━━━━━━━━━━━━━ 8s 96ms/step


Accuracy: 0.5603
Precision: 0.5697
Recall: 0.4901
F1 Score: 0.5269


In [20]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, LeakyReLU, BatchNormalization, Reshape, Conv2DTranspose, Input, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

def load_and_preprocess_data(train_dir, test_dir, img_height=128, img_width=128, batch_size=32):
    """
    Load and preprocess training and testing data
    """
    # Data augmentation for training
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    
    # Only rescaling for testing
    test_datagen = ImageDataGenerator(rescale=1./255)
    
    # Load data from directory
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary',
        color_mode='rgb'
    )
    
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary',
        color_mode='rgb',
        shuffle=False
    )
    
    return train_generator, test_generator

def build_discriminator(input_shape=(128, 128, 3)):
    """
    Build the discriminator model that will classify real vs fake images
    """
    discriminator = Sequential([
        Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=input_shape),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        
        Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        
        Conv2D(256, (5, 5), strides=(2, 2), padding='same'),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        
        Conv2D(512, (5, 5), strides=(2, 2), padding='same'),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    
    discriminator.compile(loss='binary_crossentropy', optimizer=Adam(1e-4), metrics=['accuracy'])
    return discriminator

def build_generator(latent_dim=100, output_shape=(128, 128, 3)):
    """
    Build the generator model that will create fake images
    """
    generator = Sequential([
        Dense(8*8*512, input_shape=(latent_dim,)),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Reshape((8, 8, 512)),
        
        Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        
        Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        
        Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        
        Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', activation='tanh')
    ])
    
    return generator

def build_gan(generator, discriminator):
    """
    Connect the generator and discriminator to form the GAN
    """
    discriminator.trainable = False
    gan_input = Input(shape=(100,))
    fake_image = generator(gan_input)
    gan_output = discriminator(fake_image)
    gan = Model(gan_input, gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=Adam(1e-4))
    return gan

def build_classifier():
    """
    Build a classifier for forgery detection
    """
    classifier = Sequential([
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(128, 128, 3)),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    
    classifier.compile(loss='binary_crossentropy', optimizer=Adam(1e-4), metrics=['accuracy'])
    return classifier

def train_gan_and_classifier(generator, discriminator, gan, classifier, train_generator, test_generator, 
                         latent_dim=100, epochs=50, batch_size=32, save_interval=10):
    """
    Train both the GAN and the classifier
    """
    # Create directories for saving generated images
    if not os.path.exists('gan_generated_images'):
        os.makedirs('gan_generated_images')
    
    # Get number of batches per epoch
    batches_per_epoch = len(train_generator)
    
    # Training history
    d_losses, g_losses, c_losses = [], [], []
    
    # Train the models
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        epoch_d_losses, epoch_g_losses, epoch_c_losses = [], [], []
        
        for batch in range(batches_per_epoch):
            # Get a batch of real images
            real_images, real_labels = next(train_generator)
            batch_size = real_images.shape[0]
            
            # Create noise for generator input
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            
            # Generate a batch of fake images
            generated_images = generator.predict(noise)
            
            # Train the discriminator on real and fake images
            d_loss_real = discriminator.train_on_batch(real_images, np.ones((batch_size, 1)))
            d_loss_fake = discriminator.train_on_batch(generated_images, np.zeros((batch_size, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            
            # Train the generator
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))
            
            # Train the classifier on the real images with their labels
            c_loss = classifier.train_on_batch(real_images, real_labels)
            
            # Print progress
            if batch % 20 == 0:
                print(f"Batch {batch}/{batches_per_epoch}, D Loss: {d_loss[0]}, G Loss: {g_loss}, C Loss: {c_loss[0]}")
            
            epoch_d_losses.append(d_loss[0])
            epoch_g_losses.append(g_loss)
            epoch_c_losses.append(c_loss[0])
        
        # Compute the average losses for this epoch
        d_losses.append(np.mean(epoch_d_losses))
        g_losses.append(np.mean(epoch_g_losses))
        c_losses.append(np.mean(epoch_c_losses))
        
        # Generate and save sample images
        if epoch % save_interval == 0:
            sample_noise = np.random.normal(0, 1, (16, latent_dim))
            generated_images = generator.predict(sample_noise)
            
            # Rescale from [-1, 1] to [0, 1]
            generated_images = 0.5 * generated_images + 0.5
            
            fig, axs = plt.subplots(4, 4, figsize=(10, 10))
            count = 0
            for i in range(4):
                for j in range(4):
                    axs[i, j].imshow(generated_images[count])
                    axs[i, j].axis('off')
                    count += 1
            fig.savefig(f"gan_generated_images/epoch_{epoch}.png")
            plt.close()
    
    # Plot the losses
    plt.figure(figsize=(10, 6))
    plt.plot(d_losses, label='Discriminator')
    plt.plot(g_losses, label='Generator')
    plt.plot(c_losses, label='Classifier')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('loss_history.png')
    plt.close()
    
    return d_losses, g_losses, c_losses

def evaluate_classifier(classifier, test_generator):
    """
    Evaluate the classifier on the test set
    """
    # Get test data
    test_generator.reset()
    y_true = test_generator.classes
    
    # Predict on test data
    steps = len(test_generator)
    predictions = classifier.predict(test_generator, steps=steps)
    y_pred = (predictions > 0.5).astype(int).flatten()
    
    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    # Create confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Forgery (0)', 'Authentic (1)'],
                yticklabels=['Forgery (0)', 'Authentic (1)'])
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.savefig('confusion_matrix.png')
    plt.close()
    
    # Print metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    
    return accuracy, precision, recall, f1, cm

def main():
    # Configuration
    img_height, img_width = 128, 128
    batch_size = 16
    latent_dim = 100
    epochs = 35
    
    # Define directories for your data
    # Replace these with your actual data paths
    train_dir = 'Datasets/6000/Train'  # Should have subdirectories for each class (0 and 1)
    test_dir = 'Datasets/6000/Test'    # Should have subdirectories for each class (0 and 1)
    
    # Load and preprocess data
    train_generator, test_generator = load_and_preprocess_data(
        train_dir, test_dir, img_height, img_width, batch_size
    )
    
    # Build models
    discriminator = build_discriminator((img_height, img_width, 3))
    generator = build_generator(latent_dim, (img_height, img_width, 3))
    gan = build_gan(generator, discriminator)
    classifier = build_classifier()
    
    # Train models
    train_gan_and_classifier(
        generator, discriminator, gan, classifier,
        train_generator, test_generator,
        latent_dim, epochs, batch_size
    )
    
    # Evaluate classifier
    evaluate_classifier(classifier, test_generator)
    
    # Save models
    generator.save('generator_model.h5')
    discriminator.save('discriminator_model.h5')
    classifier.save('classifier_model.h5')

if __name__ == "__main__":
    main()

Found 2991 images belonging to 2 classes.
Found 2992 images belonging to 2 classes.
Epoch 1/35
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 463ms/step
Batch 0/187, D Loss: 0.683875560760498, G Loss: 0.6875308156013489, C Loss: 0.6927975416183472
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 896ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 425ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 719ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 983ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 923ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 910ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 791ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24

c:\Users\revna\Desktop\Github\Inpainting_and_Forgery_Detn\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


187/187 ━━━━━━━━━━━━━━━━━━━━ 15s 77ms/step


Accuracy: 0.8636
Precision: 0.9713
Recall: 0.7487
F1 Score: 0.8456


In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, LeakyReLU, BatchNormalization, Reshape, Conv2DTranspose, Input, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

def load_and_preprocess_data(train_dir, test_dir, img_height=128, img_width=128, batch_size=32):
    """
    Load and preprocess training and testing data
    """
    # Data augmentation for training
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    
    # Only rescaling for testing
    test_datagen = ImageDataGenerator(rescale=1./255)
    
    # Load data from directory
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary',
        color_mode='rgb'
    )
    
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary',
        color_mode='rgb',
        shuffle=False
    )
    
    return train_generator, test_generator

def build_discriminator(input_shape=(128, 128, 3)):
    """
    Build the discriminator model that will classify real vs fake images
    """
    discriminator = Sequential([
        Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=input_shape),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        
        Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        
        Conv2D(256, (5, 5), strides=(2, 2), padding='same'),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        
        Conv2D(512, (5, 5), strides=(2, 2), padding='same'),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    
    discriminator.compile(loss='binary_crossentropy', optimizer=Adam(1e-4), metrics=['accuracy'])
    return discriminator

def build_generator(latent_dim=100, output_shape=(128, 128, 3)):
    """
    Build the generator model that will create fake images
    """
    generator = Sequential([
        Dense(8*8*512, input_shape=(latent_dim,)),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Reshape((8, 8, 512)),
        
        Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        
        Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        
        Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        
        Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', activation='tanh')
    ])
    
    return generator

def build_gan(generator, discriminator):
    """
    Connect the generator and discriminator to form the GAN
    """
    discriminator.trainable = False
    gan_input = Input(shape=(100,))
    fake_image = generator(gan_input)
    gan_output = discriminator(fake_image)
    gan = Model(gan_input, gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=Adam(1e-4))
    return gan

def build_classifier():
    """
    Build a classifier for forgery detection
    """
    classifier = Sequential([
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(128, 128, 3)),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    
    classifier.compile(loss='binary_crossentropy', optimizer=Adam(1e-4), metrics=['accuracy'])
    return classifier

def train_gan_and_classifier(generator, discriminator, gan, classifier, train_generator, test_generator, 
                         latent_dim=100, epochs=50, batch_size=32, save_interval=10):
    """
    Train both the GAN and the classifier
    """
    # Create directories for saving generated images
    if not os.path.exists('gan_generated_images'):
        os.makedirs('gan_generated_images')
    
    # Get number of batches per epoch
    batches_per_epoch = len(train_generator)
    
    # Training history
    d_losses, g_losses, c_losses = [], [], []
    
    # Train the models
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        epoch_d_losses, epoch_g_losses, epoch_c_losses = [], [], []
        
        for batch in range(batches_per_epoch):
            # Get a batch of real images
            real_images, real_labels = next(train_generator)
            batch_size = real_images.shape[0]
            
            # Create noise for generator input
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            
            # Generate a batch of fake images
            generated_images = generator.predict(noise)
            
            # Train the discriminator on real and fake images
            d_loss_real = discriminator.train_on_batch(real_images, np.ones((batch_size, 1)))
            d_loss_fake = discriminator.train_on_batch(generated_images, np.zeros((batch_size, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            
            # Train the generator
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))
            
            # Train the classifier on the real images with their labels
            c_loss = classifier.train_on_batch(real_images, real_labels)
            
            # Print progress
            if batch % 20 == 0:
                print(f"Batch {batch}/{batches_per_epoch}, D Loss: {d_loss[0]}, G Loss: {g_loss}, C Loss: {c_loss[0]}")
            
            epoch_d_losses.append(d_loss[0])
            epoch_g_losses.append(g_loss)
            epoch_c_losses.append(c_loss[0])
        
        # Compute the average losses for this epoch
        d_losses.append(np.mean(epoch_d_losses))
        g_losses.append(np.mean(epoch_g_losses))
        c_losses.append(np.mean(epoch_c_losses))
        
        # Generate and save sample images
        if epoch % save_interval == 0:
            sample_noise = np.random.normal(0, 1, (16, latent_dim))
            generated_images = generator.predict(sample_noise)
            
            # Rescale from [-1, 1] to [0, 1]
            generated_images = 0.5 * generated_images + 0.5
            
            fig, axs = plt.subplots(4, 4, figsize=(10, 10))
            count = 0
            for i in range(4):
                for j in range(4):
                    axs[i, j].imshow(generated_images[count])
                    axs[i, j].axis('off')
                    count += 1
            fig.savefig(f"gan_generated_images/epoch_{epoch}.png")
            plt.close()
    
    # Plot the losses
    plt.figure(figsize=(10, 6))
    plt.plot(d_losses, label='Discriminator')
    plt.plot(g_losses, label='Generator')
    plt.plot(c_losses, label='Classifier')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('loss_history.png')
    plt.close()
    
    return d_losses, g_losses, c_losses

def evaluate_classifier(classifier, test_generator):
    """
    Evaluate the classifier on the test set
    """
    # Get test data
    test_generator.reset()
    y_true = test_generator.classes
    
    # Predict on test data
    steps = len(test_generator)
    predictions = classifier.predict(test_generator, steps=steps)
    y_pred = (predictions > 0.5).astype(int).flatten()
    
    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    # Create confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Forgery (0)', 'Authentic (1)'],
                yticklabels=['Forgery (0)', 'Authentic (1)'])
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.savefig('confusion_matrix.png')
    plt.close()
    
    # Print metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    
    return accuracy, precision, recall, f1, cm

def main():
    # Configuration
    img_height, img_width = 128, 128
    batch_size = 16
    latent_dim = 100
    epochs = 20
    
    # Define directories for your data
    # Replace these with your actual data paths
    train_dir = "C:/Users/revna/Desktop/Github/Inpainting_and_Forgery_Detn/Datasets/6000/Train"  # Should have subdirectories for each class (0 and 1)
    test_dir = "C:/Users/revna/Desktop/Github/Inpainting_and_Forgery_Detn/Datasets/6000/Test"    # Should have subdirectories for each class (0 and 1)
    
    # Load and preprocess data
    train_generator, test_generator = load_and_preprocess_data(
        train_dir, test_dir, img_height, img_width, batch_size
    )
    
    # Build models
    discriminator = build_discriminator((img_height, img_width, 3))
    generator = build_generator(latent_dim, (img_height, img_width, 3))
    gan = build_gan(generator, discriminator)
    classifier = build_classifier()
    
    # Train models
    train_gan_and_classifier(
        generator, discriminator, gan, classifier,
        train_generator, test_generator,
        latent_dim, epochs, batch_size
    )
    
    # Evaluate classifier
    evaluate_classifier(classifier, test_generator)
    
    # Save models
    generator.save('Models/20/generator_model.h5')
    discriminator.save('Models/20/discriminator_model.h5')
    classifier.save('Models/20/classifier_model.h5')

if __name__ == "__main__":
    main()

Found 2991 images belonging to 2 classes.
Found 2992 images belonging to 2 classes.
Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 306ms/step
Batch 0/187, D Loss: 0.7017244100570679, G Loss: 0.688244104385376, C Loss: 0.676071286201477
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step

c:\Users\revna\Desktop\Github\Inpainting_and_Forgery_Detection\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


187/187 ━━━━━━━━━━━━━━━━━━━━ 13s 67ms/step


Accuracy: 0.7955
Precision: 0.8887
Recall: 0.6743
F1 Score: 0.7668


# Hyper Parameter Tuning

In [2]:
pip install keras-tuner

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, LeakyReLU, BatchNormalization, Reshape, Conv2DTranspose, Input, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import keras_tuner as kt

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

def load_and_preprocess_data(train_dir, test_dir, img_height=128, img_width=128, batch_size=32):
    """
    Load and preprocess training and testing data
    """
    # Data augmentation for training
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.2  # Add validation split for hyperparameter tuning
    )
    
    # Only rescaling for testing
    test_datagen = ImageDataGenerator(rescale=1./255)
    
    # Load data from directory
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary',
        color_mode='rgb',
        subset='training'  # Specify training subset
    )
    
    # Validation generator for hyperparameter tuning
    validation_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary',
        color_mode='rgb',
        subset='validation'  # Specify validation subset
    )
    
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary',
        color_mode='rgb',
        shuffle=False
    )
    
    return train_generator, validation_generator, test_generator

def build_discriminator(input_shape=(128, 128, 3), lr=1e-4):
    """
    Build the discriminator model that will classify real vs fake images
    """
    discriminator = Sequential([
        Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=input_shape),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        
        Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        
        Conv2D(256, (5, 5), strides=(2, 2), padding='same'),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        
        Conv2D(512, (5, 5), strides=(2, 2), padding='same'),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    
    discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr), metrics=['accuracy'])
    return discriminator

def build_generator(latent_dim=100, output_shape=(128, 128, 3)):
    """
    Build the generator model that will create fake images
    """
    generator = Sequential([
        Dense(8*8*512, input_shape=(latent_dim,)),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Reshape((8, 8, 512)),
        
        Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        
        Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        
        Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        
        Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', activation='tanh')
    ])
    
    return generator

def build_gan(generator, discriminator, lr=1e-4):
    """
    Connect the generator and discriminator to form the GAN
    """
    discriminator.trainable = False
    gan_input = Input(shape=(100,))
    fake_image = generator(gan_input)
    gan_output = discriminator(fake_image)
    gan = Model(gan_input, gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=Adam(lr))
    return gan

def build_classifier_model(hp):
    """
    Build a classifier for forgery detection with hyperparameter tuning
    """
    # Hyperparameters to tune
    conv_layers = hp.Int('conv_layers', min_value=2, max_value=4, step=1)
    filters_base = hp.Choice('filters_base', values=[16, 32, 64])
    kernel_size = hp.Choice('kernel_size', values=[3, 5])
    dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)
    dense_units = hp.Int('dense_units', min_value=128, max_value=512, step=128)
    learning_rate = hp.Choice('learning_rate', values=[1e-4, 5e-4, 1e-3])
    
    # Build model
    model = Sequential()
    
    # First convolutional block
    model.add(Conv2D(filters_base, (kernel_size, kernel_size), activation='relu', padding='same', 
                    input_shape=(128, 128, 3)))
    model.add(Conv2D(filters_base, (kernel_size, kernel_size), activation='relu', padding='same'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_rate))
    
    # Additional convolutional blocks
    for i in range(conv_layers - 1):
        filters = filters_base * (2**(i+1))
        model.add(Conv2D(filters, (kernel_size, kernel_size), activation='relu', padding='same'))
        model.add(Conv2D(filters, (kernel_size, kernel_size), activation='relu', padding='same'))
        model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(dropout_rate))
    
    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(dense_units, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(
        optimizer=Adam(learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

def build_classifier(best_hps=None):
    """
    Build a classifier for forgery detection with fixed hyperparameters
    """
    if best_hps is None:
        # Default parameters if no hyperparameter tuning was done
        classifier = Sequential([
            Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(128, 128, 3)),
            Conv2D(32, (3, 3), activation='relu', padding='same'),
            tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
            Dropout(0.25),
            
            Conv2D(64, (3, 3), activation='relu', padding='same'),
            Conv2D(64, (3, 3), activation='relu', padding='same'),
            tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
            Dropout(0.25),
            
            Conv2D(128, (3, 3), activation='relu', padding='same'),
            Conv2D(128, (3, 3), activation='relu', padding='same'),
            tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
            Dropout(0.25),
            
            Flatten(),
            Dense(512, activation='relu'),
            Dropout(0.5),
            Dense(1, activation='sigmoid')
        ])
        
        classifier.compile(loss='binary_crossentropy', optimizer=Adam(1e-4), metrics=['accuracy'])
    else:
        # Use the best hyperparameters from tuning
        conv_layers = best_hps.get('conv_layers')
        filters_base = best_hps.get('filters_base')
        kernel_size = best_hps.get('kernel_size')
        dropout_rate = best_hps.get('dropout_rate')
        dense_units = best_hps.get('dense_units')
        learning_rate = best_hps.get('learning_rate')
        
        # Build model with tuned hyperparameters
        classifier = Sequential()
        
        # First convolutional block
        classifier.add(Conv2D(filters_base, (kernel_size, kernel_size), activation='relu', padding='same', 
                          input_shape=(128, 128, 3)))
        classifier.add(Conv2D(filters_base, (kernel_size, kernel_size), activation='relu', padding='same'))
        classifier.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
        classifier.add(Dropout(dropout_rate))
        
        # Additional convolutional blocks
        for i in range(conv_layers - 1):
            filters = filters_base * (2**(i+1))
            classifier.add(Conv2D(filters, (kernel_size, kernel_size), activation='relu', padding='same'))
            classifier.add(Conv2D(filters, (kernel_size, kernel_size), activation='relu', padding='same'))
            classifier.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
            classifier.add(Dropout(dropout_rate))
        
        # Fully connected layers
        classifier.add(Flatten())
        classifier.add(Dense(dense_units, activation='relu'))
        classifier.add(Dropout(dropout_rate))
        classifier.add(Dense(1, activation='sigmoid'))
        
        # Compile model
        classifier.compile(
            optimizer=Adam(learning_rate),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
    return classifier

def hyperparameter_tuning(validation_generator, max_trials=10, epochs=5):
    """
    Perform hyperparameter tuning for the classifier model
    """
    print("Starting hyperparameter tuning...")
    
    # Create directory for tuner
    if not os.path.exists('hyperparameter_tuning'):
        os.makedirs('hyperparameter_tuning')
    
    # Initialize tuner
    tuner = kt.RandomSearch(
        build_classifier_model,
        objective='val_accuracy',
        max_trials=max_trials,
        directory='hyperparameter_tuning',
        project_name='forgery_detection'
    )
    
    # Create early stopping callback
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )
    
    # Search for best hyperparameters
    tuner.search(
        validation_generator,
        epochs=epochs,
        validation_data=validation_generator,
        callbacks=[early_stop]
    )
    
    # Get best hyperparameters
    best_hps = tuner.get_best_hyperparameters(1)[0]
    
    # Print best hyperparameters
    print("\nBest Hyperparameters:")
    print(f"Number of Conv Layers: {best_hps.get('conv_layers')}")
    print(f"Base Filters: {best_hps.get('filters_base')}")
    print(f"Kernel Size: {best_hps.get('kernel_size')}")
    print(f"Dropout Rate: {best_hps.get('dropout_rate')}")
    print(f"Dense Units: {best_hps.get('dense_units')}")
    print(f"Learning Rate: {best_hps.get('learning_rate')}")
    
    # Create a dictionary of the best hyperparameters
    best_params = {
        'conv_layers': best_hps.get('conv_layers'),
        'filters_base': best_hps.get('filters_base'),
        'kernel_size': best_hps.get('kernel_size'),
        'dropout_rate': best_hps.get('dropout_rate'),
        'dense_units': best_hps.get('dense_units'),
        'learning_rate': best_hps.get('learning_rate')
    }
    
    return best_params

def train_gan_and_classifier(generator, discriminator, gan, classifier, train_generator, validation_generator, 
                         latent_dim=100, epochs=50, batch_size=32, save_interval=10):
    """
    Train both the GAN and the classifier
    """
    # Create directories for saving generated images
    if not os.path.exists('gan_generated_images/tuned'):
        os.makedirs('gan_generated_images/tuned')
    
    # Get number of batches per epoch
    batches_per_epoch = len(train_generator)
    
    # Training history
    d_losses, g_losses, c_losses = [], [], []
    
    # Early stopping and learning rate reduction for classifier
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
    
    # Separate classifier training history for proper validation
    classifier_history = classifier.fit(
        train_generator,
        epochs=epochs,
        validation_data=validation_generator,
        callbacks=[early_stopping, reduce_lr]
    )
    
    # Extract classifier losses for plotting
    c_losses = classifier_history.history['loss']
    
    # Train the GAN models
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        epoch_d_losses, epoch_g_losses = [], []
        
        for batch in range(batches_per_epoch):
            # Get a batch of real images
            real_images, real_labels = next(train_generator)
            batch_size = real_images.shape[0]
            
            # Create noise for generator input
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            
            # Generate a batch of fake images
            generated_images = generator.predict(noise)
            
            # Train the discriminator on real and fake images
            d_loss_real = discriminator.train_on_batch(real_images, np.ones((batch_size, 1)))
            d_loss_fake = discriminator.train_on_batch(generated_images, np.zeros((batch_size, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            
            # Train the generator
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))
            
            # Print progress
            if batch % 20 == 0:
                print(f"Batch {batch}/{batches_per_epoch}, D Loss: {d_loss[0]}, G Loss: {g_loss}")
            
            epoch_d_losses.append(d_loss[0])
            epoch_g_losses.append(g_loss)
        
        # Compute the average losses for this epoch
        d_losses.append(np.mean(epoch_d_losses))
        g_losses.append(np.mean(epoch_g_losses))
        
        # Generate and save sample images
        if epoch % save_interval == 0:
            sample_noise = np.random.normal(0, 1, (16, latent_dim))
            generated_images = generator.predict(sample_noise)
            
            # Rescale from [-1, 1] to [0, 1]
            generated_images = 0.5 * generated_images + 0.5
            
            fig, axs = plt.subplots(4, 4, figsize=(10, 10))
            count = 0
            for i in range(4):
                for j in range(4):
                    axs[i, j].imshow(generated_images[count])
                    axs[i, j].axis('off')
                    count += 1
            fig.savefig(f"gan_generated_images/epoch_{epoch}.png")
            plt.close()
    
    # Plot the losses
    plt.figure(figsize=(10, 6))
    plt.plot(d_losses, label='Discriminator')
    plt.plot(g_losses, label='Generator')
    plt.plot(c_losses, label='Classifier')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('loss_history.png')
    plt.close()
    
    # Plot classifier metrics
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(classifier_history.history['accuracy'], label='Train Accuracy')
    plt.plot(classifier_history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(classifier_history.history['loss'], label='Train Loss')
    plt.plot(classifier_history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('classifier_training_history.png')
    plt.close()
    
    return d_losses, g_losses, c_losses

def evaluate_classifier(classifier, test_generator):
    """
    Evaluate the classifier on the test set
    """
    # Get test data
    test_generator.reset()
    y_true = test_generator.classes
    
    # Predict on test data
    steps = len(test_generator)
    predictions = classifier.predict(test_generator, steps=steps)
    y_pred = (predictions > 0.5).astype(int).flatten()
    
    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    # Create confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Forgery (0)', 'Authentic (1)'],
                yticklabels=['Forgery (0)', 'Authentic (1)'])
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.savefig('confusion_matrix.png')
    plt.close()
    
    # Print metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    
    return accuracy, precision, recall, f1, cm

def main():
    # Configuration
    img_height, img_width = 128, 128
    batch_size = 16
    latent_dim = 100
    epochs = 35
    
    # Define directories for your data
    # Replace these with your actual data paths
    train_dir = "C:/Users/revna/Desktop/Github/Inpainting_and_Forgery_Detn/Datasets/6000/Train"  # Should have subdirectories for each class (0 and 1)
    test_dir = "C:/Users/revna/Desktop/Github/Inpainting_and_Forgery_Detn/Datasets/6000/Test"    # Should have subdirectories for each class (0 and 1)
    
    # Load and preprocess data
    train_generator, validation_generator, test_generator = load_and_preprocess_data(
        train_dir, test_dir, img_height, img_width, batch_size
    )
    
    # Create directory for models
    if not os.path.exists('Models/20'):
        os.makedirs('Models/20')
    
    # Perform hyperparameter tuning
    print("Starting hyperparameter tuning for classifier...")
    best_params = hyperparameter_tuning(validation_generator, max_trials=15, epochs=5)
    
    # Build models with tuned hyperparameters
    discriminator_lr = best_params.get('learning_rate', 1e-4)
    discriminator = build_discriminator((img_height, img_width, 3), lr=discriminator_lr)
    generator = build_generator(latent_dim, (img_height, img_width, 3))
    gan = build_gan(generator, discriminator, lr=discriminator_lr)
    classifier = build_classifier(best_params)
    
    # Print model summaries
    print("\nGenerator Model Summary:")
    generator.summary()
    
    print("\nDiscriminator Model Summary:")
    discriminator.summary()
    
    print("\nClassifier Model Summary:")
    classifier.summary()
    
    # Train models
    print("\nStarting model training...")
    train_gan_and_classifier(
        generator, discriminator, gan, classifier,
        train_generator, validation_generator,
        latent_dim, epochs, batch_size
    )
    
    # Evaluate classifier
    print("\nEvaluating classifier on test set...")
    evaluate_classifier(classifier, test_generator)
    
    # Save models
    print("\nSaving models...")
    generator.save('Models/tuned/generator_model.h5')
    discriminator.save('Models/tuned/discriminator_model.h5')
    classifier.save('Models/tuned/classifier_model.h5')
    
    # Save hyperparameters
    np.save('Models/tuned/best_hyperparameters.npy', best_params)
    
    print("Training and evaluation complete!")

if __name__ == "__main__":
    main()

Found 2393 images belonging to 2 classes.
Found 598 images belonging to 2 classes.
Found 2992 images belonging to 2 classes.
Starting hyperparameter tuning for classifier...
Starting hyperparameter tuning...
Reloading Tuner from hyperparameter_tuning\forgery_detection\tuner0.json

Best Hyperparameters:
Number of Conv Layers: 2
Base Filters: 32
Kernel Size: 3
Dropout Rate: 0.5
Dense Units: 256
Learning Rate: 0.0005


c:\Users\revna\Desktop\Github\Inpainting_and_Forgery_Detection\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\revna\Desktop\Github\Inpainting_and_Forgery_Detection\venv\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
c:\Users\revna\Desktop\Github\Inpainting_and_Forgery_Detection\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Generator Model Summary:


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_1 (Dense)                 │ (None, 32768)          │     3,309,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32768)          │       131,072 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 8, 8, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 16, 16, 256)    │     3,277,056 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 16, 16, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_5 (LeakyReLU)       │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 32, 32, 128)    │       819,328 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_6 (LeakyReLU)       │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_2              │ (None, 64, 64, 64)     │       204,864 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 64, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_7 (LeakyReLU)       │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_3              │ (None, 128, 128, 3)    │         4,803 │
│ (Conv2DTranspose)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

c:\Users\revna\Desktop\Github\Inpainting_and_Forgery_Detection\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


150/150 ━━━━━━━━━━━━━━━━━━━━ 34s 213ms/step - accuracy: 0.4975 - loss: 0.8073 - val_accuracy: 0.4983 - val_loss: 0.6932 - learning_rate: 5.0000e-04
Epoch 2/35
150/150 ━━━━━━━━━━━━━━━━━━━━ 33s 219ms/step - accuracy: 0.4907 - loss: 0.6934 - val_accuracy: 0.5000 - val_loss: 0.6933 - learning_rate: 5.0000e-04
Epoch 3/35
150/150 ━━━━━━━━━━━━━━━━━━━━ 33s 217ms/step - accuracy: 0.4908 - loss: 0.6938 - val_accuracy: 0.5318 - val_loss: 0.6931 - learning_rate: 5.0000e-04
Epoch 4/35
150/150 ━━━━━━━━━━━━━━━━━━━━ 36s 239ms/step - accuracy: 0.5079 - loss: 0.6938 - val_accuracy: 0.5017 - val_loss: 0.6930 - learning_rate: 5.0000e-04
Epoch 5/35
150/150 ━━━━━━━━━━━━━━━━━━━━ 39s 263ms/step - accuracy: 0.4900 - loss: 0.6940 - val_accuracy: 0.5017 - val_loss: 0.6931 - learning_rate: 5.0000e-04
Epoch 6/35
150/150 ━━━━━━━━━━━━━━━━━━━━ 40s 267ms/step - accuracy: 0.5035 - loss: 0.6931 - val_accuracy: 0.5017 - val_loss: 0.6932 - learning_rate: 5.0000e-04
Epoch 7/35
150/150 ━━━━━━━━━━━━━━━━━━━━ 41s 270ms/step - 

c:\Users\revna\Desktop\Github\Inpainting_and_Forgery_Detection\venv\Lib\site-packages\keras\src\backend\tensorflow\trainer.py:82: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Batch 0/150, D Loss: 0.6998269557952881, G Loss: 0.7117516994476318
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
Batch 20/150, D Loss: 0.7012010812759399, G Loss: 0.49077823758125305
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 

Accuracy: 0.6340
Precision: 0.7118
Recall: 0.4471
F1 Score: 0.5492

Saving models...
Training and evaluation complete!
